In [1]:
import json
import pandas as pd

# مسیر فایل دیتا (فعلاً فرضی)
data_path = "../data/raw/data.json"

print("Notebook is running ✅")


data_path = "../data/raw/data.json"

with open(data_path, encoding="utf-8") as f:
    data = json.load(f)

print("نوع داده:", type(data))
print("تعداد رکورد:", len(data))


Notebook is running ✅
نوع داده: <class 'dict'>
تعداد رکورد: 4


In [2]:
data.keys()


dict_keys(['name', 'type', 'id', 'messages'])

In [3]:
type(data['messages'])


list

In [4]:
len(data['messages'])


4596

In [5]:
data['messages'][0]


{'id': 1,
 'type': 'service',
 'date': '2021-09-03T13:57:38',
 'date_unixtime': '1630661258',
 'actor': 'معرفی اساتید علموص',
 'actor_id': 'channel1558862150',
 'action': 'create_channel',
 'title': 'معرفی اساتید علموص',
 'text': '',
 'text_entities': []}

In [6]:
for msg in data['messages']:
    if msg.get('type') == 'message':
        msg
        break


In [7]:
messages = data['messages']

valid_messages = []

for msg in messages:
    if msg.get('type') != 'message':
        continue
    
    text = msg.get('text')
    if not text:
        continue
    
    if isinstance(text, list):
        text = ''.join(
            part if isinstance(part, str) else ''
            for part in text
        )
    
    valid_messages.append({
        'date': msg.get('date'),
        'text': text
    })

len(valid_messages)


4556

In [8]:
valid_messages[:3]


[{'date': '2021-09-05T00:24:19',
  'text': '\n\nسلام به همه\u200cی علموصی ها😍👊🏾\n\n✅این ربات برای فرآیند  آماده شده.\n\n📚تموم اطلاعاتی که راجع به اساتید نیازه، \nمثل:  ،  ،  و ... رو میتونین از این کانال بدست بیارید.\n\n🆔 \n🆔 \n\nهمچنین شما هم میتونید اطلاعات اساتیدی که باهاشون کلاس داشتین رو ثبت کنید تا دوستانتون استفاده کنند👇\n\n🤖 '},
 {'date': '2021-09-05T00:34:39',
  'text': '🧑\u200d🏫 سید محمد شهرتاش\n🏫 \n📒 بررسی ۱-بررسی۲-حفاظت\n\nمنابع آموزش\n  ┘ فایل پاورپوینت یا پی دی اف می دهد\n\nحضور و غیاب\n  ┘ حضور و غیاب نمی کند\n\nمنابع معرفی شده برای امتحان کافی است؟\n  ┘ بله\n\nوضعیت نمره دادن:\n  ┘ منصفانه و هرچی خودت بگیری\n\nارزیابی دانشجو:\n  ┤ پیوستگی و یکپارچگی تدریس: 8\n  ┤ دانش عمومی: 9\n  ┤ توانایی انتقال مطالب و نکات مهم درس: 7\n  ┤ نحوه مدیریت کلاس(نظم و زمان): 9\n  ┤ پاسخگویی(حضوری و غیرحضوری): 8\n  ┤ آداب و رفتار اجتماعی با دانشجویان: 10\n\nراه ارتباطی:\n  ┘ چیزی ندارم\n\nترمی که دانشجو با این استاد کلاس داشته:\n  ┘ مهر 99\n\nتوضیحات:\n  ┘ چیزی اضافه ایی نیست\n~~~~~~~~~~~~~~

In [9]:
import re

structured_data = []

for item in valid_messages:
    text = item['text']

    # فقط پیام‌هایی که واقعاً معرفی استاد هستن
    if 'ارزیابی دانشجو' not in text:
        continue

    professor = None
    course = None
    scores = {}

    # استخراج نام استاد
    prof_match = re.search(r'🧑‍🏫\s*(.+)', text)
    if prof_match:
        professor = prof_match.group(1).strip()

    # استخراج درس
    course_match = re.search(r'📒\s*(.+)', text)
    if course_match:
        course = course_match.group(1).strip()

    # استخراج امتیازها
    score_patterns = {
        'teaching': r'انتقال مطالب.*?:\s*(\d+)',
        'knowledge': r'دانش عمومی.*?:\s*(\d+)',
        'management': r'مدیریت کلاس.*?:\s*(\d+)',
        'response': r'پاسخگویی.*?:\s*(\d+)',
        'behavior': r'آداب و رفتار.*?:\s*(\d+)',
        'coherence': r'پیوستگی.*?:\s*(\d+)'
    }

    for key, pattern in score_patterns.items():
        m = re.search(pattern, text)
        scores[key] = int(m.group(1)) if m else None

    structured_data.append({
        'date': item['date'],
        'professor': professor,
        'course': course,
        **scores
    })

len(structured_data)


4276

In [10]:
structured_data[:3]


[{'date': '2021-09-05T00:34:39',
  'professor': 'سید محمد شهرتاش',
  'course': 'بررسی ۱-بررسی۲-حفاظت',
  'teaching': 7,
  'knowledge': 9,
  'management': 9,
  'response': 8,
  'behavior': 10,
  'coherence': 8},
 {'date': '2021-09-05T00:35:51',
  'professor': 'دکتر هاجر فلاحتی',
  'course': 'مدار منطقی- طراحی سیستم های کامپیوتری',
  'teaching': 1,
  'knowledge': 2,
  'management': 3,
  'response': 2,
  'behavior': 10,
  'coherence': 1},
 {'date': '2021-09-05T00:36:56',
  'professor': 'علم الهدی',
  'course': 'بلور شناسی',
  'teaching': 10,
  'knowledge': 8,
  'management': 9,
  'response': 10,
  'behavior': 10,
  'coherence': 9}]